# Creating a Dummy GPT Model

In [5]:
# We will use this configuration:
from re import M


GPT_CONFIG_124M = {
    "vocab_size": 50257,  # Vocabulary size used by BPE tokenizer
    "context_length": 1024, # max numbers of vocabulary the token can handle via positional embeddings
    "emb_dim": 768, # embedding size for each vocab
    "n_heads": 12, # count of attention heads in multi-head attention mechanism
    "n_layers": 12, # 
    "drop_rate": 0.1, # intensity dropout mechanism, 0.1 means 10% to prevent overfittin g
    "qkv_bias": False # we initially disable this
}

In [8]:
import torch
import torch.nn as nn

class DummyGPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # now we create the token embedding inside class (not outside)
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        self.trf_blocks = nn.Seqential(
            *[DummyTransformerBlock(cfg) for _ in range(cfg["n_layers"])]
        )
        self.final_norm = DummyLayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

class DummyTransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
    def forward(self, x):
        return x 
    
class DummyLayerNorm(nn.Module):
    def __init__(self, normalized_shape, eps=1e-5):
        super().__init__()
    def forward(self, x):
        return x
